In [48]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
#from transformers import BertForSequenceClassification, BertTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

In [49]:
# Ruta a Carpeta de Codigos de Estudiantes y al modelo CodeBERTa
ruta_carpeta = 'ruta\carpeta\ejemplos'
ruta_modelo = 'ruta\modelo\preentrenado'

In [50]:
# Inicialización de DataFrame
data = pd.DataFrame()

In [51]:
# Cargar el modelo pre-entrenado
model = AutoModelForSequenceClassification.from_pretrained(ruta_modelo)
tokenizer = AutoTokenizer.from_pretrained(ruta_modelo)

# Cargar el modelo pre-entrenado y el tokenizador
#model = BertForSequenceClassification.from_pretrained(ruta_modelo, num_labels=num_etiquetas)
#tokenizer = BertTokenizer.from_pretrained(ruta_modelo)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at D:\joaqu\Material UNAB\Tesis\CodeBERTa and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [52]:
# Función de preprocesamiento

def preprocesar_codigo(codigo):
    # Limpiar código fuente
    codigo = codigo.replace('...', '')  # Eliminar comentarios
    codigo = codigo.replace(' ', ' ')  # Eliminar espacios en blanco

    # Tokenizar código fuente
    tokens = word_tokenize(codigo)

    # Eliminar tokens irrelevantes
    stop_words = set(stopwords.words('spanish'))
    tokens = [token for token in tokens if token not in stop_words]

    return tokens

In [53]:
# Función de vectorización y almacenamiento

def vectorizar_y_almacenar(tokens_list):
    # Vectorizar el código con CodeBERTa
    representaciones = []

    for tokens in tokens_list:
        inputs = tokenizer(tokens, return_tensors='pt')  # Codificar el código
        with torch.no_grad():
            outputs = model(**inputs)  # Obtener representaciones del modelo
        representaciones.append(outputs.logits.squeeze(0).detach().cpu().numpy())  # Extraer representaciones

    # Almacenar datos en DataFrame - FOCO DE ERROR
    data_vectorizada = pd.DataFrame(representaciones, columns=model.config.num_labels_to_classify)
    return data_vectorizada

In [54]:
# Funciones de KNN
def entrenar_modelo_knn(X_train, y_train, k):
    # Entrenar el modelo kNN
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    return knn

def predecir_calificaciones(knn, X_test):
    # Predecir las calificaciones de los estudiantes
    y_pred = knn.predict(X_test)
    return y_pred

# Funciones de agrupamiento
def agrupar_estudiantes(data, columna_agrupamiento):
    # Agrupar estudiantes por columna especificada
    grupos = data.groupby(columna_agrupamiento)

    # Obtener la información de cada grupo
    grupos_info = grupos.agg({
        'codigo_estudiante': 'unique',
        'calificacion': 'mean'
    }).reset_index()

    # Renombrar columnas
    grupos_info.rename(columns={'codigo_estudiante': 'estudiantes', 'calificacion': 'calificacion_promedio'}, inplace=True)

    return grupos_info

In [55]:
# Recorrer carpeta de codigos
tokens_list = []

for archivo in os.listdir(ruta_carpeta):
    if archivo.endswith('.py'):
        # Leer el archivo
        with open(os.path.join(ruta_carpeta, archivo), 'r') as f:
            codigo = f.read()

        # Preprocesar el código
        tokens = preprocesar_codigo(codigo)
        tokens_list.append(tokens)

In [61]:
# Vectorizar y almacenar
data_vectorizada = vectorizar_y_almacenar(tokens_list)

# Unir datos con DataFrame principal
data = data.join(data_vectorizada)

AttributeError: 'RobertaConfig' object has no attribute 'num_labels_to_classify'

In [ ]:
# Visualización de resultados
data.groupby('calificacion')['codigo_preprocesado'].apply(list).head()

In [ ]:
# Parámetros de KNN
k = 5  # Definir el valor de k

# Entrenamiento del modelo kNN
X_train, X_test, y_train, y_test = train_test_split(data_vectorizada, data['calificacion'], test_size=0.2)
knn = entrenar_modelo_knn(X_train, y_train, k)

In [ ]:
# Predicción de calificaciones
y_pred = predecir_calificaciones(knn, X_test)

In [ ]:
# Agrupamiento de estudiantes
columna_agrupamiento = 'grupo'  # Definir la columna de agrupamiento
grupos_info = agrupar_estudiantes(data, columna_agrupamiento)